In [ ]:
# ----- NOTES -----
# - pg.85: most common supervised learning tasks are regression and classification
# - in this chapter we are using the MNIST dataset, what is this?
# 

In [ ]:
# ----- IMPORTS -----

In [ ]:
# here we are fetching 